In [3]:
import os, numpy as np, pandas as pd
try:
    from gurobipy import Model, GRB, quicksum
except Exception as e:
    raise RuntimeError("Please install and license Gurobi (gurobipy) before running this cell.") from e

# Paths 
BASE_DIR = os.path.expanduser("~/Downloads/IEOR4004EProject1")
PATH_REG = os.path.join(BASE_DIR, "child_care_regulated.csv")
PATH_POP = os.path.join(BASE_DIR, "population.csv")
PATH_INC = os.path.join(BASE_DIR, "avg_individual_income.csv")
PATH_EMP = os.path.join(BASE_DIR, "employment_rate.csv")

# Policy / cost parameters
EXPANSION_COST_PER_SLOT = 250.0           # per-slot expansion cost (applies to all added slots)
EQUIP_COST_0_5_PER_SLOT = 100.0           # equipment surcharge per 0–5 slot (new + expanded)
NEW_COST   = {"S": 65000.0, "M": 95000.0, "L": 115000.0}
NEW_TOTAL  = {"S": 100, "M": 200, "L": 400}
NEW_0_5_MAX = {"S": 50,  "M": 100, "L": 200}
NEW_5_12_MAX = {k: NEW_TOTAL[k] - NEW_0_5_MAX[k] for k in NEW_TOTAL}

# facility-level expansion rule
BASELINE_FIXED_IF_100PCT = 20000.0        # fixed part if expand >= 100% at a facility
BASELINE_PER_EXISTING    = 200.0          # per existing slot if expand >= 100%
MAX_EXP_FRACTION         = 1.20           # up to +120% of existing facility capacity
MAX_EXP_SLOTS_PER_FAC    = 500.0          # hard cap per facility
BIG_M                    = 500.0          # safe Big-M (matches MAX_EXP_SLOTS_PER_FAC)
EPS                      = 1e-6

# Load data
reg = pd.read_csv(PATH_REG)
pop = pd.read_csv(PATH_POP)
inc = pd.read_csv(PATH_INC)
emp = pd.read_csv(PATH_EMP)

# ZIP rule: take only the first 5 digits 
def _zip_trunc(val):
    s = ''.join(ch for ch in str(val) if ch.isdigit())
    return np.nan if not s else int(s[:5])

reg["zipcode"] = reg["zip_code"].apply(_zip_trunc)
pop["zipcode"] = pop["zipcode"].apply(_zip_trunc)
inc["zipcode"] = inc["ZIP code"].apply(_zip_trunc)         
emp["zipcode"] = emp["zipcode"].apply(_zip_trunc)

for d in (reg, pop, inc, emp):
    d.dropna(subset=["zipcode"], inplace=True)
    d["zipcode"] = d["zipcode"].astype(int)

# Aggregate existing capacity by ZIP
# 0–5 existing = infant + toddler + preschool; older existing = school_age
reg["cap_0_5"]  = reg[["infant_capacity","toddler_capacity","preschool_capacity"]].sum(axis=1)
reg["cap_5_12"] = reg[["school_age_capacity"]].sum(axis=1)

by_zip = (reg.groupby("zipcode")
            .agg(cap0_5=("cap_0_5","sum"),
                 cap5_12=("cap_5_12","sum"),
                 total_cap=("total_capacity","sum"),
                 n_facilities=("facility_id","count"))
            .reset_index())

# Build ZIP table with socio-econ & population
df = (by_zip
      .merge(pop[["zipcode","-5","5-9","10-14"]], on="zipcode", how="left")
      .merge(inc[["zipcode","average income"]],   on="zipcode", how="left")
      .merge(emp[["zipcode","employment rate"]],  on="zipcode", how="left"))

df[["cap0_5","cap5_12","total_cap"]] = df[["cap0_5","cap5_12","total_cap"]].fillna(0.0)
df["n_facilities"] = df["n_facilities"].fillna(0).astype(int)

# Non-overlapping population groups
''' Assumption to avoid double counting:
 0–5  := (-5) + 1/5*(5–9)      -> adds 5-year-olds
 5–9  := 4/5*(5–9)             -> ages 6–9 only
 9–12 := 3/5*(10–14)           -> ages 10–12 only'''
df["pop_0_5"]   = df["-5"].fillna(0.0) + 0.2 * df["5-9"].fillna(0.0)
df["pop_5_9"]   = 0.8 * df["5-9"].fillna(0.0)
df["pop_9_12"]  = 0.6 * df["10-14"].fillna(0.0)
df["pop_0_12"]  = df["pop_0_5"] + df["pop_5_9"] + df["pop_9_12"]

# High-demand classification
df["employment rate"]   = df["employment rate"].fillna(0.0)
df["average income"]    = df["average income"].fillna(1e9)
df["is_high_demand"]    = ((df["employment rate"] >= 0.60) | (df["average income"] <= 60000)).astype(int)

# Thresholds (strictly above desert line)
df["threshold_total"]   = np.where(df["is_high_demand"]==1, 0.50*df["pop_0_12"], (1/3.0)*df["pop_0_12"])
df["threshold_total"]   = np.ceil(df["threshold_total"] + 1.0)
df["threshold_0_5"]     = np.ceil((2.0/3.0)*df["pop_0_5"])

# Facility list for expansion decisions
fac = reg[["facility_id","zipcode","total_capacity"]].copy()
fac = fac.dropna(subset=["facility_id","zipcode"]).copy()
# ensure types
fac["facility_id"] = fac["facility_id"].astype(str)
fac["zipcode"] = fac["zipcode"].astype(int)
fac["total_capacity"] = pd.to_numeric(fac["total_capacity"], errors="coerce").fillna(0.0)

# Map: ZIP -> list of facility_ids
zip_to_facs = fac.groupby("zipcode")["facility_id"].apply(list).to_dict()
cap_by_fac  = fac.set_index("facility_id")["total_capacity"].to_dict()

# Model (Gurobi)
m = Model("IEOR4004E_Idealistic_Gurobi_facility_expansion")

# New facility counts (integers)
yS = {z: m.addVar(vtype=GRB.INTEGER, lb=0, name=f"yS[{z}]") for z in df["zipcode"]}
yM = {z: m.addVar(vtype=GRB.INTEGER, lb=0, name=f"yM[{z}]") for z in df["zipcode"]}
yL = {z: m.addVar(vtype=GRB.INTEGER, lb=0, name=f"yL[{z}]") for z in df["zipcode"]}

# New capacity allocations by age group (continuous, planning-level)
u_new_0_5  = {z: m.addVar(vtype=GRB.CONTINUOUS, lb=0.0, name=f"u_new_0_5[{z}]")  for z in df["zipcode"]}
u_new_5_9  = {z: m.addVar(vtype=GRB.CONTINUOUS, lb=0.0, name=f"u_new_5_9[{z}]")  for z in df["zipcode"]}
u_new_9_12 = {z: m.addVar(vtype=GRB.CONTINUOUS, lb=0.0, name=f"u_new_9_12[{z}]") for z in df["zipcode"]}

# facility-level expansion variables by age group
eF_0_5   = {f: m.addVar(vtype=GRB.CONTINUOUS, lb=0.0, name=f"eF_0_5[{f}]")   for f in cap_by_fac}
eF_5_9   = {f: m.addVar(vtype=GRB.CONTINUOUS, lb=0.0, name=f"eF_5_9[{f}]")   for f in cap_by_fac}
eF_9_12  = {f: m.addVar(vtype=GRB.CONTINUOUS, lb=0.0, name=f"eF_9_12[{f}]")  for f in cap_by_fac}
# Check if expansion >= 100% at facility f
yF_100p  = {f: m.addVar(vtype=GRB.BINARY, name=f"yF_100p[{f}]")               for f in cap_by_fac}

m.update()

def total_new(z):     return NEW_TOTAL["S"]*yS[z] + NEW_TOTAL["M"]*yM[z] + NEW_TOTAL["L"]*yL[z]
def max_new_0_5(z):   return NEW_0_5_MAX["S"]*yS[z] + NEW_0_5_MAX["M"]*yM[z] + NEW_0_5_MAX["L"]*yL[z]
def max_new_5_12(z):  return NEW_5_12_MAX["S"]*yS[z] + NEW_5_12_MAX["M"]*yM[z] + NEW_5_12_MAX["L"]*yL[z]

# Facility-level expansion limits & trigger
for f, C_f in cap_by_fac.items():
    E_f = eF_0_5[f] + eF_5_9[f] + eF_9_12[f]
    # Capacity limit: up to +120% of existing, and capped at 500 slots
    m.addConstr(E_f <= MAX_EXP_FRACTION * C_f, name=f"exp_frac_cap[{f}]")
    m.addConstr(E_f <= MAX_EXP_SLOTS_PER_FAC,  name=f"exp_abs_cap[{f}]")
    # Trigger yF_100p = 1 iff E_f >= C_f  (two linear constraints)
    m.addConstr(E_f - C_f * yF_100p[f] >= 0.0,                   name=f"trigger_lb[{f}]")
    m.addConstr(E_f <= (C_f - EPS) + BIG_M * yF_100p[f],         name=f"trigger_ub[{f}]")

# ZIP-level coverage constraints (use facility sums)
dfi = df.set_index("zipcode")

for z, r in dfi.iterrows():
    # Sum facility expansions in this ZIP
    facs = zip_to_facs.get(int(z), [])
    sum_e0_5  = quicksum(eF_0_5[f]  for f in facs)
    sum_e5_9  = quicksum(eF_5_9[f]  for f in facs)
    sum_e9_12 = quicksum(eF_9_12[f] for f in facs)

    total_after_0_5  = r["cap0_5"]  + sum_e0_5 + u_new_0_5[z]
    total_after_older= r["cap5_12"] + (sum_e5_9 + sum_e9_12) + (u_new_5_9[z] + u_new_9_12[z])
    total_after      = total_after_0_5 + total_after_older

    # Coverage
    m.addConstr(total_after     >= r["threshold_total"], name=f"not_desert[{z}]")
    m.addConstr(total_after_0_5 >= r["threshold_0_5"],  name=f"u05_policy[{z}]")

    # New build allocation envelopes
    m.addConstr(u_new_0_5[z]                 <= max_new_0_5(z),  name=f"new_0_5_cap[{z}]")
    m.addConstr(u_new_5_9[z] + u_new_9_12[z] <= max_new_5_12(z), name=f"new_older_cap[{z}]")
    m.addConstr(u_new_0_5[z] + u_new_5_9[z] + u_new_9_12[z] == total_new(z), name=f"new_alloc_bal[{z}]")

# Build + equipment(0–5) + expansion per-slot + baseline if >=100% 
build_cost  = quicksum(NEW_COST["S"]*yS[z] + NEW_COST["M"]*yM[z] + NEW_COST["L"]*yL[z] for z in dfi.index)
equip_cost  = quicksum(EQUIP_COST_0_5_PER_SLOT * (u_new_0_5[z]) for z in dfi.index) \
            + quicksum(EQUIP_COST_0_5_PER_SLOT * (eF_0_5[f])    for f in cap_by_fac)  # equipment also for 0–5 expansions
expand_slots= quicksum(eF_0_5[f] + eF_5_9[f] + eF_9_12[f]       for f in cap_by_fac)
expand_cost = EXPANSION_COST_PER_SLOT * expand_slots

# One-time baseline cost if expansion >= 100% at a facility 
baseline_cost = quicksum((BASELINE_FIXED_IF_100PCT + BASELINE_PER_EXISTING * cap_by_fac[f]) * yF_100p[f]
                         for f in cap_by_fac)

m.setObjective(build_cost + equip_cost + expand_cost + baseline_cost, GRB.MINIMIZE)

m.Params.OutputFlag = 1
m.optimize()

# Save solution
rows = []
for z, r in dfi.iterrows():
    y_s, y_m, y_l = int(round(yS[z].X)), int(round(yM[z].X)), int(round(yL[z].X))
    new_total = NEW_TOTAL["S"]*y_s + NEW_TOTAL["M"]*y_m + NEW_TOTAL["L"]*y_l

    # Sum facility expansions in ZIP for reporting
    facs = zip_to_facs.get(int(z), [])
    sum_e0_5  = sum(eF_0_5[f].X  for f in facs)
    sum_e5_9  = sum(eF_5_9[f].X  for f in facs)
    sum_e9_12 = sum(eF_9_12[f].X for f in facs)

    rows.append(dict(
        zipcode=int(z),
        is_high_demand=int(r["is_high_demand"]),
        pop_0_5=float(r["pop_0_5"]), pop_5_9=float(r["pop_5_9"]), pop_9_12=float(r["pop_9_12"]),
        pop_0_12=float(r["pop_0_12"]),
        threshold_total=float(r["threshold_total"]), threshold_0_5=float(r["threshold_0_5"]),
        existing_cap_total=float(r["total_cap"]),
        existing_cap_0_5=float(r["cap0_5"]),
        existing_cap_5_12=float(r["cap5_12"]),
        y_small=y_s, y_medium=y_m, y_large=y_l,
        u_new_0_5=float(u_new_0_5[z].X),
        u_new_5_9=float(u_new_5_9[z].X),
        u_new_9_12=float(u_new_9_12[z].X),
        e_0_5=float(sum_e0_5),
        e_5_9=float(sum_e5_9),
        e_9_12=float(sum_e9_12),
        new_total_slots=float(new_total)
    ))

# Facility-level audit to determine which facilities triggered the baseline
fac_rows = []
for f, C_f in cap_by_fac.items():
    fac_rows.append(dict(
        facility_id=f,
        zipcode=int(fac.loc[fac["facility_id"]==f, "zipcode"].iloc[0]),
        existing_total=float(C_f),
        exp_0_5=float(eF_0_5[f].X),
        exp_5_9=float(eF_5_9[f].X),
        exp_9_12=float(eF_9_12[f].X),
        expanded_slots=float(eF_0_5[f].X + eF_5_9[f].X + eF_9_12[f].X),
        triggered_baseline=int(round(yF_100p[f].X))
    ))
zip_df = pd.DataFrame(rows).sort_values("zipcode")
fac_df = pd.DataFrame(fac_rows).sort_values(["zipcode","facility_id"])

OUT1 = os.path.join(BASE_DIR, "idealistic_solution_by_zip.csv")
OUT2 = os.path.join(BASE_DIR, "idealistic_expansion_by_facility.csv")
zip_df.to_csv(OUT1, index=False)
fac_df.to_csv(OUT2, index=False)
print("\n Objective (USD) =", f"{m.objVal:,.2f}")
print("Wrote:", OUT1)
print("Wrote:", OUT2)

Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 68356 rows, 69544 columns and 303443 nonzeros
Model fingerprint: 0x65415ad6
Variable types: 50376 continuous, 19168 integer (15604 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+02]
  Objective range  [1e+02, 2e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-06, 1e+04]
Presolve removed 68334 rows and 69512 columns
Presolve time: 0.81s
Presolved: 22 rows, 32 columns, 91 nonzeros
Variable types: 18 continuous, 14 integer (11 binary)
Found heuristic solution: objective 3.423426e+08
Found heuristic solution: objective 3.423237e+08

Root relaxation: objective 3.422076e+08, 11 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | 